## Deliverable 2. Create a Customer Travel Destinations Map.

In [16]:
# Dependencies and Setup
import os
import pandas as pd
import requests
import gmaps
import numpy as np


# Import API key
from config3 import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv(os.path.join("..", "Weather_Database", "WeatherPy_database.csv"))
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Havoysund,NO,2022-02-07 00:04:18,70.9963,24.6622,25.68,79,37,16.69
1,1,Vaini,TO,2022-02-07 00:08:33,-21.2000,-175.2000,77.16,94,75,14.97
2,2,Port Lincoln,AU,2022-02-07 00:08:34,-34.7333,135.8667,77.85,49,0,16.93
3,3,Hilo,US,2022-02-07 00:05:07,19.7297,-155.0900,78.53,76,100,11.50
4,4,Saint-Philippe,RE,2022-02-07 00:08:34,-21.3585,55.7679,76.59,83,99,13.31


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [11]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
1,1,Vaini,TO,2022-02-07 00:08:33,-21.2000,-175.2000,77.16,94,75,14.97
2,2,Port Lincoln,AU,2022-02-07 00:08:34,-34.7333,135.8667,77.85,49,0,16.93
3,3,Hilo,US,2022-02-07 00:05:07,19.7297,-155.0900,78.53,76,100,11.50
4,4,Saint-Philippe,RE,2022-02-07 00:08:34,-21.3585,55.7679,76.59,83,99,13.31
5,5,Rikitea,PF,2022-02-07 00:08:35,-23.1203,-134.9692,79.32,71,3,18.54
8,8,Kahului,US,2022-02-07 00:08:35,20.8947,-156.4700,82.42,59,20,26.46
13,13,Atuona,PF,2022-02-07 00:08:37,-9.8000,-139.0333,78.76,76,20,18.41
14,14,Port Moresby,PG,2022-02-07 00:08:37,-9.4431,147.1797,83.05,67,100,5.75
19,19,Puerto Ayora,EC,2022-02-07 00:03:42,-0.7393,-90.3518,75.54,90,36,4.00
20,20,Grand Gaube,MU,2022-02-07 00:08:38,-20.0064,57.6608,77.38,83,90,1.99


In [12]:
# 4a. Determine if there are any empty rows.
print(len(preferred_cities_df))
preferred_cities_df.count()

173


City_ID       173
City          173
Country       171
Date          173
Lat           173
Lng           173
Max Temp      173
Humidity      173
Cloudiness    173
Wind Speed    173
dtype: int64

In [13]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
pref_cities_df = preferred_cities_df.dropna(axis="index", how="any")
print(len(pref_cities_df))
pref_cities_df.head()

171


,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
1,1,Vaini,TO,2022-02-07 00:08:33,-21.2000,-175.2000,77.16,94,75,14.97
2,2,Port Lincoln,AU,2022-02-07 00:08:34,-34.7333,135.8667,77.85,49,0,16.93
3,3,Hilo,US,2022-02-07 00:05:07,19.7297,-155.0900,78.53,76,100,11.50
4,4,Saint-Philippe,RE,2022-02-07 00:08:34,-21.3585,55.7679,76.59,83,99,13.31
5,5,Rikitea,PF,2022-02-07 00:08:35,-23.1203,-134.9692,79.32,71,3,18.54


In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = pref_cities_df[["City", "Country", "Max Temp", "Cloudiness", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Cloudiness,Lat,Lng,Hotel Name
1,Vaini,TO,77.16,75,-21.2000,-175.2000,
2,Port Lincoln,AU,77.85,0,-34.7333,135.8667,
3,Hilo,US,78.53,100,19.7297,-155.0900,
4,Saint-Philippe,RE,76.59,99,-21.3585,55.7679,
5,Rikitea,PF,79.32,3,-23.1203,-134.9692,
8,Kahului,US,82.42,20,20.8947,-156.4700,
13,Atuona,PF,78.76,20,-9.8000,-139.0333,
14,Port Moresby,PG,83.05,100,-9.4431,147.1797,
19,Puerto Ayora,EC,75.54,36,-0.7393,-90.3518,
20,Grand Gaube,MU,77.38,90,-20.0064,57.6608,


In [17]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found...skipping the city.")
        hotel_df.loc[index, "Hotel Name"] = np.nan
        

Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.


In [18]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna(axis="index", how="any")
clean_hotel_df.head(10)

,City,Country,Max Temp,Cloudiness,Lat,Lng,Hotel Name
1,Vaini,TO,77.16,75,-21.2000,-175.2000,Keleti Beach Resort
2,Port Lincoln,AU,77.85,0,-34.7333,135.8667,Port Lincoln Hotel
3,Hilo,US,78.53,100,19.7297,-155.0900,Hilo Hawaiian Hotel
4,Saint-Philippe,RE,76.59,99,-21.3585,55.7679,"Chambres d'hôte ""La Trinité"""
5,Rikitea,PF,79.32,3,-23.1203,-134.9692,People ThankYou
8,Kahului,US,82.42,20,20.8947,-156.4700,Maui Seaside Hotel
13,Atuona,PF,78.76,20,-9.8000,-139.0333,Villa Enata
14,Port Moresby,PG,83.05,100,-9.4431,147.1797,Lamana Hotel (Port Moresby)
19,Puerto Ayora,EC,75.54,36,-0.7393,-90.3518,Finch Bay Galapagos Hotel
20,Grand Gaube,MU,77.38,90,-20.0064,57.6608,Veranda Paul et Virginie Hotel & Spa


In [19]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [20]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Cloudiness}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [21]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))